In [131]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from joblib import Parallel, delayed
from dtw import *
from fastdtw import fastdtw # scales linearly so might be better when using larger arrays, but slower for shorter ones and approximates

----
Test runtimes for different packages

In [132]:
# time dtw from package
# https://pypi.org/project/dtw-python/
def example_usage():
    S = np.random.rand(80, 50)  # Example spectrogram S
    T = np.random.rand(85, 50)  # Example spectrogram T

    alignment = dtw(S, T, keep_internals=True)
    #print("Distance:", alignment)
    #print("Cumulative Distance Matrix:\n", D)

In [133]:
%timeit example_usage()

1.05 ms ± 54.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [134]:
# time dtw from package
# https://pypi.org/project/fastdtw/
def example_usage():
    S = np.random.rand(80, 50)  # Example spectrogram S
    T = np.random.rand(85, 50)  # Example spectrogram T

    dist, path = fastdtw(S, T, radius=30, dist=euclidean)
    #print("Distance:", dist)
    #print("Cumulative Distance Matrix:\n", D)

In [135]:
%timeit example_usage()

125 ms ± 11.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


------
Perform dtfw using the package first on the time axis, then as usual

In [136]:
def compute_dfw_matrix(S1, S2, parallel = False, fast=True):
    n1 = S1.shape[0] # number of frequency bins (time steps)
    n2 = S2.shape[0]

    # Function to compute DTW for a single pair of spectra
    def compute_dfw_pair(i, j, fast):
        s1_bin = S1[i, :]
        s2_bin = S2[j, :]
        if fast:
            dist, _ = fastdtw(s1_bin, s2_bin, radius=30, dist=euclidean)
            return i, j, dist
        alignment = dtw(s1_bin, s2_bin)
        return i, j, alignment.normalizedDistance # TODO replace with distance? (not normalized)

    if parallel:
    # Compute DTW for all pairs of columns in parallel
        results = Parallel(n_jobs=-1)(
            delayed(compute_dfw_pair)(i, j, fast) for i in range(n1) for j in range(n2)
        )

    else:
        results = []
        for i in range(n1): 
            for j in range(n2):
                results.append(compute_dfw_pair(i, j, fast))

    # Fill the DTW matrix
    dfw_matrix = np.zeros((n1, n2))
    for i, j, distance in results:
        dfw_matrix[i, j] = distance

    return dfw_matrix

In [137]:
# S1.shape = (M, F)
# S2.shape = (N, F)

def dtfw(S1, S2, parallel=True, fast=True):
    if not S1.shape[1] == S2.shape[1]:
        print("Unequal length of frequency bins")
        return None
    
    # Compute matrix with cumulative distances per spectrum combination
    dfw_matrix = compute_dfw_matrix(S1, S2, parallel=parallel, fast=fast)

    # Conventional DTW using the new matrix holding the cumulative distance per spectra combination
    # if fast:
    #     return fastdtw(dfw_matrix, dist=euclidean)
    
    alignment = dtw(dfw_matrix)
    return alignment


----
Test runtime for custom code, using both parallel cumputing and vanilla

In [138]:
def example_usage(nt1, nt2, nf, parallel=True, fast=False):
    S = np.random.rand(nt1, nf)  # Example spectrogram S
    T = np.random.rand(nt2, nf)  # Example spectrogram T

    alignment = dtfw(S, T, parallel=parallel, fast=fast)
    #print("Distance:", dist)
    #print("Cumulative Distance Matrix:\n", D)

In [139]:
%timeit example_usage(80, 85, 50)

631 ms ± 18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [140]:
%timeit example_usage(80, 85, 50, parallel=False)

3.43 s ± 62.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [141]:
%timeit example_usage(800, 850, 100)

KeyboardInterrupt: 